In [35]:
%matplotlib inline

import cPickle, gzip

import numpy as np
import matplotlib.pyplot as plt

import theano
from theano import shared, pp, grad, function
from theano import tensor as T

In [36]:
f = gzip.open('mnist.pkl.gz','rb')
train_set,valid_set,test_set = cPickle.load(f)
f.close()

In [37]:
def share_dataset(data_xy):
    data_x,data_y = data_xy
    shared_x = shared(np.array(data_x,dtype=theano.config.floatX))
    shared_y = shared(np.array(data_y,dtype=theano.config.floatX))
    return shared_x,T.cast(shared_y,'int32')

train_set_x,train_set_y = share_dataset(train_set)
valid_set_x,valid_set_y = share_dataset(valid_set)
test_set_x,test_set_y = share_dataset(test_set)

In [54]:
class LogisticRegression(object):
    def __init__(self,x,n_in,n_out):
        self.W = theano.shared(
            value=np.zeros(
                (n_in,n_out),
                dtype=theano.config.floatX
            ),
            name='W',
            borrow=True
        )
        self.b = theano.shared(
            value=np.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )
        self.p_y_given_x = T.nnet.softmax(T.dot(x,self.W)+self.b)
        self.y_pred = T.argmax(self.p_y_given_x,axis=1)
        self.params = [ self.W, self.b ]
        self.x = x
    
    def negative_log_likehood(self,y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]),y])
    
    def errors(self,y):
        return T.mean(T.neq(self.y_pred,y))

In [82]:
batch_size = 500
n_train_batchs = train_set_x.get_value(borrow=True).shape[0]/batch_size
n_valid_batchs = valid_set_x.get_value(borrow=True).shape[0]/batch_size
n_test_batchs = test_set_x.get_value(borrow=True).shape[0]/batch_size

i = T.lscalar('i')
x = T.matrix('x')
y = T.ivector('y')

classifier = LogisticRegression(x=x,n_in=28*28,n_out=10)
cost = classifier.negative_log_likehood(y)

test_model = function(
    inputs=[i],
    outputs=[classifier.errors(y)],
    givens=[
        (x, test_set_x[i*batch_size:(i+1)*batch_size]),
        (y, test_set_y[i*batch_size:(i+1)*batch_size])
    ]
)

valid_model = function(
    inputs=[i],
    outputs=[classifier.errors(y)],
    givens=[
        (x, valid_set_x[i*batch_size:(i+1)*batch_size]),
        (y, valid_set_y[i*batch_size:(i+1)*batch_size])
    ]
)

alpha = 0.5
g_W = grad(cost=cost,wrt=classifier.W)
g_b = grad(cost=cost,wrt=classifier.b)

updates = [
    (classifier.W, classifier.W - alpha*g_W),
    (classifier.b, classifier.b - alpha*g_b),
]

train_model = function(
    inputs=[i],
    outputs=[cost],
    updates=updates,
    givens=[
        (x, train_set_x[i*batch_size:(i+1)*batch_size]),
        (y, train_set_y[i*batch_size:(i+1)*batch_size])
    ]
)

In [83]:
print test_model(0)
for epoch in range(1000):
    costs = [] 
    for i in range(n_train_batchs):
        costs.append(train_model(i))
#     print np.mean(costs)
print test_model(0)

[array(0.916)]
[array(0.06)]
